## Imports

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
from scipy.misc import imread, imresize

In [2]:
from keras.models import load_model
from keras.applications.mobilenet import relu6, DepthwiseConv2D

Using TensorFlow backend.


In [3]:
model = load_model('mobilenet_finetune.hdf5', custom_objects={
                   'relu6': relu6,
                   'DepthwiseConv2D': DepthwiseConv2D})

## Data

In [4]:
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
ori_dir = os.path.join(data_dir, 'ori')
ori_csv = os.path.join(ori_dir, 'orientations.csv')

In [5]:
os.chdir(ori_dir)
num = 360
files = sorted(glob("*.png"))
x = np.asarray([imresize(imread(file,mode='RGB'), (128,128)) for file in files[:num]])
df = pd.read_csv(ori_csv,delimiter=',')
os.chdir(base_dir)

/home/nathan/anaconda3/envs/DL-Keras/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/nathan/anaconda3/envs/DL-Keras/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [6]:
df['roomba_rad'] = np.deg2rad(df['roomba_rotation'])
df['roomba_xhat'] = np.cos(df['roomba_rad'])
df['roomba_yhat'] = np.sin(df['roomba_rad'])

In [7]:
y = df[:num][['roomba_xhat', 'roomba_yhat']].values

In [8]:
y_deg = df[:num]['roomba_rotation'].values

## Compute predictions

In [9]:
def vecToAngle(vec):
    return np.degrees(np.arctan2(vec[1], vec[0]))

In [10]:
preds = model.predict(x, 32)

In [11]:
preds = [x % 360 for x in list(map(vecToAngle, preds))]

In [12]:
preds

[357.85001826286316,
 356.83843421936035,
 357.83149480819702,
 0.89474284648895264,
 0.025325929746031761,
 3.3457560539245605,
 12.557384490966797,
 5.4926247596740723,
 4.9182872772216797,
 7.8207759857177734,
 10.834826469421387,
 10.578956604003906,
 13.675079345703125,
 14.137688636779785,
 16.136354446411133,
 21.40855598449707,
 21.325441360473633,
 13.468524932861328,
 16.977920532226562,
 18.501144409179688,
 22.8271484375,
 24.93499755859375,
 25.29625129699707,
 18.199163436889648,
 28.247673034667969,
 30.308956146240234,
 25.924394607543945,
 27.486509323120117,
 27.650722503662109,
 34.050270080566406,
 30.411108016967773,
 31.633876800537109,
 31.80223274230957,
 33.963741302490234,
 32.772953033447266,
 32.425197601318359,
 36.887741088867188,
 37.855552673339844,
 39.503631591796875,
 40.402519226074219,
 41.198200225830078,
 43.663200378417969,
 40.239837646484375,
 44.891777038574219,
 51.072341918945312,
 43.992694854736328,
 48.102790832519531,
 46.976436614990234

## Animate Predictions

In [13]:
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML, Image